# Run_Correlation_Assays

This notebook is necessary as running the shell script '`Correlation_Assays.sh`' doesn't seem to work as expected.

In [1]:
from IPython import nbformat
from IPython.nbconvert.preprocessors import ExecutePreprocessor

In [2]:
# ChEMBL connection...

engine = create_engine(open('database.txt').read().strip())

In [3]:
# Load symbols...

df = pd.read_sql_query("""
    select
          symbol
        , count(*) as n
    from
        tt_curve_data_v1
    group by 
        symbol
    order by
          n desc
        , symbol
    """, engine)

df.shape

(185, 2)

In [4]:
# Monkey-patch ExecutePreprocessor to throw exception if executing a cell fails
# For details, see https://github.com/ipython/ipython/issues/8286

def preprocess_cell(self, cell, resources, cell_index):
    """
    Apply a transformation on each code cell. See base.py for details.
    """
    if cell.cell_type != 'code':
        return cell, resources

    outputs = self.run_cell(cell)
    cell.outputs = outputs

    for out in outputs:
        if out.output_type == 'error':
            logging.error('Error executing the following cell:')
            logging.error(str(cell.source))
            raise RuntimeError('Error executing the notebook.')

    return cell, resources

ExecutePreprocessor.preprocess_cell = preprocess_cell

In [5]:
# Load template notebook...

nb = nbformat.read('Correlation_Assays.ipynb', as_version=4)

output_dir = 'Correlation_Assays'

In [ ]:
# Remaining symbols to process (simple way to allow manual restart)...

# symbols = df['symbol'].tolist()

# open('symbols_to_do.txt', 'w').write('\n'.join(symbols) + '\n')

symbols = open('symbols_to_do.txt').read().split()

In [9]:
# Run template notebook for each symbol...

for symbol in symbols:

    logging.info("Starting symbol '{}'...".format(symbol))
    
    nb_outfile = '{}/{}.ipynb'.format(output_dir, symbol)
        
    os.environ['symbol'] = symbol
        
    try:

        nb_out, _ = ExecutePreprocessor(timeout=3600).preprocess(nb, {})
        
        success = True
        
        logging.info("...'{}' succeeded.".format(symbol))

    except RuntimeError as e:

        if e.message != 'Error executing the notebook.': raise
            
        success = False

        logging.warn("...'{}' failed.".format(symbol))
        
    ### success = not any(x['outputs'][0].get('ename') == 'AssertionError' for x in nb_out['cells'] if x.get('outputs'))
        
    if success:
        
        nbformat.write(nb_out, nb_outfile)

[2015/Aug/07 12:13:38 INFO    ] Starting symbol 'HRH3'...
[2015/Aug/07 12:15:57 INFO    ] ...'HRH3' succeeded.
[2015/Aug/07 12:15:57 INFO    ] Starting symbol 'SLC6A2'...
[2015/Aug/07 12:17:38 INFO    ] ...'SLC6A2' succeeded.
[2015/Aug/07 12:17:38 INFO    ] Starting symbol 'F2'...
[2015/Aug/07 12:19:40 ERROR   ] Error executing the following cell:
[2015/Aug/07 12:19:40 ERROR   ] assert subset.shape[0] > 0, "No pairs of assays with sufficient compounds in common in interesting subset"
[2015/Aug/07 12:19:41 WARNING ] ...'F2' failed.
[2015/Aug/07 12:19:41 INFO    ] Starting symbol 'ADORA3'...
[2015/Aug/07 12:22:39 INFO    ] ...'ADORA3' succeeded.
[2015/Aug/07 12:22:39 INFO    ] Starting symbol 'OPRK1'...
[2015/Aug/07 12:25:12 ERROR   ] Error executing the following cell:
[2015/Aug/07 12:25:12 ERROR   ] assert subset.shape[0] > 0, "No pairs of assays with sufficient compounds in common in interesting subset"
[2015/Aug/07 12:25:12 WARNING ] ...'OPRK1' failed.
[2015/Aug/07 12:25:12 INFO    ]

In [31]:
# Add to table of symbols...

def f(symbol):
    
    file_name = "{}/{}.ipynb".format(output_dir, symbol)
        
    return '<a target="_blank" href="{}">link</a>'.format(file_name) if os.path.exists(file_name) else None

df['report'] = df['symbol'].apply(f)

In [34]:
df[~df['report'].isnull()]

,symbol,n,report
22,ACHE,4777,link
8,ADORA1,7570,link
12,ADORA2A,6393,link
19,ADORA3,4988,link
29,ADRA1A,3763,link
53,ADRA1D,1746,link
78,ADRA2A,1005,link
94,ADRA2B,705,link
47,ADRB2,1945,link
119,CACNA1C,177,link
